Author: Nguyen Thanh Quan. 2022-07-16
Repository: 

This notebook represents an approach to find an optimal subset of sensors from a number of original physical ones. It is about how missing values of a sensor can be imputed again based on its high correlated sensors. The correlation among sensors is calculated with Pearson.

Dataset: solar energy
Missing rate: 30% for the sensor SolarPowerAarhus:486 (faulty sensor)

In [7]:
# Necessary packages
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import yaml
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pgain.pgain import pgain
from libs.gainutils import normalization
from libs.utils import transform_missing_data, load_full_data, calulate_pearson_corrs
from kneed import KneeLocator

Global vars

In [8]:
ROOT_PATH = ROOT_PATH = os.getcwd()
params = yaml.load(open(ROOT_PATH + '/config.yaml'), yaml.Loader)

Load data and define params

In [9]:
# Load data and define params
gain_parameters = {'batch_size': 128,
                    'hint_rate': 0.9,
                    'alpha': 100,
                    'iterations': 10000,
                    'data_type': 'solar',
                    'miss_rate': 0.3}
    
data_type = 'solar'
miss_rate = 0.2

# Get list of sensors
data_frame = load_full_data(data_type, True)

# Rename to consolidate the name of the sensors
data_frame.rename(columns={'response': 'predictor0'}, inplace=True)

Loaded historic data from: urn_ngsi-ld_Sensor_SolarPowerAarhus_473_currentProduction.csv with 24000 samples
Loaded historic data from: urn_ngsi-ld_Sensor_SolarPowerAarhus_474_currentProduction.csv with 24000 samples
Loaded historic data from: urn_ngsi-ld_Sensor_SolarPowerAarhus_475_currentProduction.csv with 24000 samples
Loaded historic data from: urn_ngsi-ld_Sensor_SolarPowerAarhus_476_currentProduction.csv with 24000 samples
Loaded historic data from: urn_ngsi-ld_Sensor_SolarPowerAarhus_477_currentProduction.csv with 24000 samples
Loaded historic data from: urn_ngsi-ld_Sensor_SolarPowerAarhus_478_currentProduction.csv with 24000 samples
Loaded historic data from: urn_ngsi-ld_Sensor_SolarPowerAarhus_479_currentProduction.csv with 24000 samples
Loaded historic data from: urn_ngsi-ld_Sensor_SolarPowerAarhus_480_currentProduction.csv with 24000 samples
Loaded historic data from: urn_ngsi-ld_Sensor_SolarPowerAarhus_481_currentProduction.csv with 24000 samples
Loaded historic data from: u

Calculate correlation

In [10]:
# Loop to calculate imputed data
results_k_sens = pd.DataFrame(columns=['rank', 'predicted_sensor', 'n_discarded_sensors', 'ecdf_area_95th'])
all_sensors = list(data_frame)
sensor_map_pearson_ranks = calulate_pearson_corrs(data_frame)

ranks = {
    'pearson':sensor_map_pearson_ranks
}

rank_labels={
    'pearson': 'Pearson'
}

less_corr_sensors = []

Find optimal subset

In [11]:
for idx, sens in enumerate(all_sensors):
    sensor_corrs = sensor_map_pearson_ranks[sens]
    # sensor_corrs = sorted(sensor_corrs, key=lambda x: x[1], reverse=False) # in this way, we have the sensors from the worst to the best
    corr_values = [x[1] for x in sensor_corrs if x[1] > 0.6]

    # if len(corr_values) < 8:
    #     print("Less correlated sensors: {}, skipping...".format(idx))
    #     less_corr_sensors.append(idx)
    #     continue

    position = 0

    # Rearrange to predict
    predicted_column = data_frame.pop(sens)
    data_frame.insert(position, sens, predicted_column)

    for k in range(position, len(all_sensors) - position - 1):
        _data_frame = data_frame.iloc[: , :-k]
        
        if k == 0:
            _data_frame = data_frame.copy()

        # print(_data_frame)
        
        ori_data_x, miss_data_x, data_m = transform_missing_data(_data_frame, miss_rate, sens)
        imputed_pos = int(ori_data_x.shape[0] * (1 - miss_rate))

        if len(corr_values) >= params['less_sensors']:
            # Impute missing data
            imputed_data_x = pgain(miss_data_x, gain_parameters)

            ori_data, norm_parameters = normalization(ori_data_x)
            imputed_data, _ = normalization(imputed_data_x, norm_parameters)

            # Extract only predicted sensor
            ori_predicted_col = [col[position] for col in ori_data]
            imputed_data_col = [col[position] for col in imputed_data]

            # Calculate 95 error
            errors = np.abs(np.asarray(ori_predicted_col[imputed_pos:]) - np.asarray(imputed_data_col[imputed_pos:]))
        
        else:
            print("Less correlated sensors: {}, highest error by default...".format(sens))
            errors = np.ones((ori_data_x.shape[0] - imputed_pos))

        # print("Errors: {}".format(errors))
        # errors = rmse_loss(ori_data_x, imputed_data_x, data_m)
        ecdf_x = np.sort(errors)
        ecdf_y = np.arange(1, len(ecdf_x) ) / len(ecdf_x)
        perc_95 = int(len(ecdf_x)*0.95)

        ecdf_area_95 = np.trapz(ecdf_y[:perc_95], ecdf_x[:perc_95])

        new_row = pd.DataFrame()
        new_row['rank'] = ['pearson']
        new_row['predicted_sensor'] = [sens]
        new_row['n_discarded_sensors'] = [k]
        new_row['ecdf_area_95th'] = [ecdf_area_95]
        results_k_sens = results_k_sens.append(new_row)

# Start drawing plot
plt.figure(figsize=(20, 8))
plot_string = "Area under the curve: \n"

ranks_errors_list = []
for key in ranks:
    subresults = results_k_sens.query("rank == @key")
    grouped = subresults.groupby(['n_discarded_sensors']).sum()
    auc = round(np.trapz(grouped['ecdf_area_95th'], list(range(0, len(grouped['ecdf_area_95th'])))), 2)
    ranks_errors_list += [(auc, key)]

ranks_errors_list = sorted(ranks_errors_list, reverse=False)
maxval = -1

for key in [ k for (_,k) in ranks_errors_list ]:
    subresults = results_k_sens.query("rank == @key")
    grouped = subresults.groupby(['n_discarded_sensors']).sum()

    maxval = np.max(grouped['ecdf_area_95th']) if np.max(grouped['ecdf_area_95th']) > maxval else maxval

    plt.plot(range(1, len(grouped['ecdf_area_95th']) + 1), grouped['ecdf_area_95th'].tolist()[::-1], label=rank_labels[key])
    
    plot_string += "  > " + rank_labels[key] + ": " + str(round(np.trapz(grouped['ecdf_area_95th'], list(range(0, len(grouped['ecdf_area_95th'])))), 2)) + "\n"

plt.xlabel("Number of selected sensors (from best to worst according to the rank)")
plt.ylabel("Sum of 95th percentile error: {}".format(data_type))
plt.legend(title="Ranking strategy")
plt.text(x=7.2, y=maxval-1.79, s=plot_string)
plt.xticks(np.arange(1, len(grouped['ecdf_area_95th']) + 1, 1.0))
plt.show()



predictor0 is selected to predict with missing rate: 0.2 ...

Total: (22970, 21)
Number of missing values: 4594
Less correlated sensors: predictor0, highest error by default...

predictor0 is selected to predict with missing rate: 0.2 ...

Total: (22970, 20)
Number of missing values: 4594
Less correlated sensors: predictor0, highest error by default...

predictor0 is selected to predict with missing rate: 0.2 ...

Total: (22970, 19)
Number of missing values: 4594
Less correlated sensors: predictor0, highest error by default...

predictor0 is selected to predict with missing rate: 0.2 ...

Total: (22970, 18)
Number of missing values: 4594
Less correlated sensors: predictor0, highest error by default...

predictor0 is selected to predict with missing rate: 0.2 ...

Total: (22970, 17)
Number of missing values: 4594
Less correlated sensors: predictor0, highest error by default...

predictor0 is selected to predict with missing rate: 0.2 ...

Total: (22970, 16)
Number of missing values: 45

2022-07-25 22:14:22.866498: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
100%|██████████| 10000/10000 [00:23<00:00, 433.52it/s]



predictor11 is selected to predict with missing rate: 0.2 ...

Total: (22970, 20)
Number of missing values: 4594


100%|██████████| 10000/10000 [00:20<00:00, 479.38it/s]



predictor11 is selected to predict with missing rate: 0.2 ...

Total: (22970, 19)
Number of missing values: 4594


...


predictor11 is selected to predict with missing rate: 0.2 ...

Total: (22970, 17)
Number of missing values: 4594


100%|██████████| 10000/10000 [00:26<00:00, 373.72it/s]



predictor11 is selected to predict with missing rate: 0.2 ...

Total: (22970, 16)
Number of missing values: 4594


 60%|██████    | 6004/10000 [00:16<00:10, 370.33it/s]


KeyboardInterrupt: 

Borda voting to find sensors' weight

In [ ]:
# Borda vorting
results_best_sens = {}
for s in all_sensors:
    results_best_sens[s] = 0

n_refs = len(all_sensors)
distance = 'pearson'
rank = ranks[distance]
epsilon = params['epsilon']
sensors_error = results_k_sens.query('rank==@distance').groupby('predicted_sensor')[['ecdf_area_95th']].sum().values
sensors_weight = np.subtract(1, np.divide(sensors_error, np.max(sensors_error+epsilon)))

for i, sens in enumerate(all_sensors):
    sensors_rank = rank[sens]
    sensors_rank = sorted(sensors_rank, key=lambda x: x[1], reverse=False) # in this way, we have the sensors from the best to the worst

    sensors_rank = [x for (x, _) in sensors_rank[0:n_refs]]

    # first j sensors in the rank
    for j in range(n_refs - 1):
        if j in less_corr_sensors:
            continue
        results_best_sens[sensors_rank[j]]+=(n_refs-j)*sensors_weight[i][0]

plt.figure(figsize=(14,4))

results_best_sens=sorted([x for x in zip(results_best_sens.keys(), results_best_sens.values())], key=lambda pair: pair[1], reverse=True)

keys=[x[0] for x in results_best_sens]
values=[x[1] for x in results_best_sens]


kn = KneeLocator(range(len(values)), values, curve='convex', direction='decreasing')
print('elbow:', kn.knee + 1)

plt.bar(range(len(values)), height=values)
plt.plot(range(len(values)), values, color='b')

plt.xticks(range(len(keys)), keys, rotation=35, ha="right", rotation_mode="anchor")
plt.ylabel('Weighted Borda count vote')# 'Borda count vote considering '+str(n_refs)+' positions'
plt.show()


UFuncTypeError: ufunc 'add' did not contain a loop with signature matching types (dtype('<U32'), dtype('<U32')) -> dtype('<U32')